# Validate "Generate Post" Backend Flow

This notebook exercises the FastAPI backend's `POST /posts/generate` flow and basic retrieval endpoints.

It assumes the API is running locally at `http://localhost:4000` (see `backend-py/README.md`). If you set an `API_KEY` in `.env`, the notebook will include it automatically from your environment.


In [ ]:
import os
import requests
from pprint import pprint

BASE_URL = os.getenv('LPG_BASE_URL', 'http://localhost:4000')
API_KEY = os.getenv('API_KEY', '')
HEADERS = { 'Content-Type': 'application/json' }
if API_KEY:
    HEADERS['x-api-key'] = API_KEY

def show(title, obj):
    print(f'\n=== {title} ===')
    pprint(obj)

print('BASE_URL =', BASE_URL)
print('Using API_KEY =', 'yes' if API_KEY else 'no')


## 1) Health check
Call `GET /health` to ensure the API is reachable.


In [ ]:
r = requests.get(f'{BASE_URL}/health', headers=HEADERS, timeout=10)
r.raise_for_status()
show('Health', r.json())


## 2) Generate a draft
Call `POST /posts/generate` with an optional topic. The backend will use the ReAct agent (Anthropic + Perplexity tool) if configured, otherwise gracefully fall back to the stub.


In [ ]:
payload = { 'topic': 'AI productivity' }  # You can change or set to {}
r = requests.post(f'{BASE_URL}/posts/generate', json=payload, headers=HEADERS, timeout=90)
print('Status:', r.status_code)
data = r.json()
show('Generated Post', data)
post_id = data.get('id')
assert post_id, 'No id returned'

# Basic shape assertions
required_keys = ['id','name','idea','title','text','imageUrl','status','createdAt','updatedAt']
missing = [k for k in required_keys if k not in data]
assert not missing, f'Missing keys in response: {missing}'
assert data['status'] == 'draft', f"Expected status 'draft', got {data['status']}"


## 3) List drafts
Confirm the draft is persisted by calling `GET /posts?status=draft`.


In [ ]:
r = requests.get(f'{BASE_URL}/posts', params={'status':'draft'}, headers=HEADERS, timeout=10)
r.raise_for_status()
drafts = r.json()
print('Draft count:', len(drafts))
# Show the first 1-2 drafts
show('First drafts', drafts[:2])
assert any(p.get('id') == post_id for p in drafts), 'Generated post not found in drafts list'


## 4) Get by ID
Verify we can retrieve the same draft by its `id` using `GET /posts/{id}`.


In [ ]:
r = requests.get(f'{BASE_URL}/posts/{post_id}', headers=HEADERS, timeout=10)
r.raise_for_status()
by_id = r.json()
show('Post by ID', by_id)
assert by_id.get('id') == post_id, 'Mismatched id'


---
### Notes
- If you configured `API_KEY` in `backend-py/.env`, export it in your shell so this notebook picks it up.
- Without API keys, the backend still works with stubs (LLM, research, image).
- To run multiple generations, re-run the generation cell or wrap it in a loop.
